# RSA cryptosystem
Intuition: it is feasible to find integers $d, e, n$ such that
$$ (m^e)^d\equiv m \pmod{n}$$
for integers $m<n$, such that it is infeasible to find $d$ given $e$, $n$, and/or $m$, and that
$$ (m^d)^e\equiv m \pmod{n}.$$


In [1]:
from Crypto.Util.number import getPrime, inverse, size
import secrets
import math

In [2]:
def new_key(key_bits:int, e:int = 65537):
    p = getPrime(key_bits//2)
    q = getPrime(key_bits//2)

    # public key:
    n = p*q    
    
    # totient function
    phi = (p-1)*(q-1)
    # private key:
    d = inverse(e, phi)
    
    return(e, n, d)

In [3]:
def encrypt(plaintext:int, exponent:int, public_key:int) -> int:

    ciphertext = pow(plaintext, exponent, public_key)
    
    return ciphertext

In [4]:
def decrypt(ciphertext:int, private_key:int, public_key:int) -> int:

    plaintext = pow(ciphertext, private_key, public_key)
    
    return plaintext

In [5]:
def new_flag(flag_bytes:int):
    with open('/usr/share/dict/words') as w:
		
        numWords = flag_bytes
        words = [word.strip() for word in w]

        flag = ' '.join(secrets.choice(words) for i in range(numWords))
    return flag[0:flag_bytes]

In [6]:
key_bits = 256
(e,n,d) = new_key(key_bits)
flag_bytes = 30
flag = new_flag(flag_bytes)

# encode the string as an integer
f = int.from_bytes(bytes(flag, 'ascii'), byteorder='big')
ciphertext = encrypt(f, e, n)

plaintext = decrypt(ciphertext, d, n)
# decode the plaintext as string
p = plaintext.to_bytes(key_bits//8, byteorder='big').decode('ascii')

## Special cases

In [ ]:
print(encrypt(0,e,n))
print(encrypt(1,e,n))
print(encrypt(n,e,n))
print(encrypt(-1,e,n))

## Attacks
---
### Small exponent $e$  - plaintext recovery
Suppose $e=3$. For a small plaintext $m$, it is possible to brute-force guess $m$: $$\sqrt[e]{c}$$

In [40]:
import gmpy2
message = 'secret'
m = int.from_bytes(bytes(message, 'ascii'), byteorder='big')
c = pow(m, 3, n)
print(f'modulus: {n}')
print(f'ciphertext: {c}')
cube_root = gmpy2.iroot_rem(c,3)
print(f'cube root(c): {cube_root}') # root and remainder
print(f'guess: {cube_root[0]}')
print(f'original m: {m}')

modulus: 52483463755050358431291594167767225838979059379795538692642750521578587312439
ciphertext: 2042548109113812252163525474272795974844736
cube root(c): (mpz(126879297332596), mpz(0))
guess: 126879297332596
original m: 126879297332596


If $e$ is small but $m^3>n$, it may be possible to recover $m$ from small multiples: $$\sqrt[e]{c+kn}$$

In [46]:
message = 'supr secret'
m = int.from_bytes(bytes(message, 'ascii'), byteorder='big')
c_before_modulo = pow(m,3)
print(f'pow(m,3) before modulo: (unkown without private key) \n{c_before_modulo}')
c = pow(m, 3, n)
print(f'modulus: \n{n}')
print(f'ciphertext: {c}')
cube_root = gmpy2.iroot_rem(c,3)
print(f'cube root(c): {cube_root}') # root and remainder
print(f'guess: {cube_root[0]}')
print(f'original m: {m}')

for k in range(100):
    guess = int(gmpy2.iroot_rem(c+k*n,3)[0])
    confirmed = pow(guess, 3, n) == c
    if confirmed:
        print(f'match found for k={k}')
        break

pow(m,3) before modulo: (unkown without private key) 
2719439991958273208733653161439045609027040839550764719847986258704299828330816
modulus: 
52483463755050358431291594167767225838979059379795538692642750521578587312439
ciphertext: 42783340450704928737781858882917091239108811181192246523205982103791875396427
cube root(c): (mpz(34975040831413158487309332), mpz(876629958637493208307685442414314476310612321154059))
guess: 34975040831413158487309332
original m: 139581060393214157926851956
match found for k=51


---
## Factoring $n$ - private key recovery
_Factoring attack.__

The private key $d$ is constructed from the prime factors of $n$ via the totient function:
$$
\phi(n) = (p-1)\cdot(q-1)\\
d = e^{-1}\pmod{\phi(n)}
$$
Factoring $n$ allows immediate computation of $d$.

Integer factoring options: look-up or computation
- [factordb](http://factordb.com/) or [wolframalpha](https://www.wolframalpha.com/input/?i=factor+)
- [pyecm](https://github.com/martingkelly/pyecm) using [gmpy2](https://github.com/aleaxit/gmpy)

In [10]:
import pyecm

P = getPrime(32)
Q = getPrime(32)
N = P*Q
verbose = False
random_sigma = True
asymptotic_speed = 7
processing_power = 1.0
lf = list(pyecm.factors(N, verbose, random_sigma, asymptotic_speed, processing_power))
print(f'factors of {N}: prod {lf} = {math.prod(lf)}')

factors of 14036266608999809879: prod [mpz(3292142069), mpz(4263566491)] = 14036266608999809879


Factorization difficulty depends on the best known algorithm. For recommendations, see [SP 800-56b](https://csrc.nist.gov/publications/detail/sp/800-56b/rev-2/final).

---
## Common factor
_Factoring attack._

Suppose moduli $m,n$ were generated with one prime factor $p$ in common; then $p=\gcd{m,n}$.

$\gcd$ is an extremely quick computation; large keys offer no protection.

See [Lenstra et al](https://eprint.iacr.org/2012/064) _"Ron was wrong, Whit is right"_ and [Heninger et al.](https://www.usenix.org/conference/usenixsecurity12/technical-sessions/presentation/heninger) _"Mining Your Ps and Qs: Detection of Widespread Weak Keys in Network Devices"_.

In [18]:
p = getPrime(2048)
q = getPrime(2048)
r = getPrime(2048)
m = p*q
n = p*r
print(f'gcd(m,n) = \n{math.gcd(m,n)}')
print(f'check: p = \n{p}')

gcd(m,n) = 
26815956581423103359639388739227375486729327191521064439200612917557997203718666616757319132554692571254242871845056948911524055741393408414547394728532939737063906306374966054809787745097906319910098341727407960065665932461953594734532739281068816094247662586446983966700564071573064794281207090993409366310421449669017375310616218791884204603262015374412502350564937378184614399667905384018091494990034604102596699991805461415642234374511170303138658889725032172951944175723585418771518725562386375279109772853235976421016335356522385658663402742849258298417380050438233360033352679664291994273951617250507168434043
check: p = 
2681595658142310335963938873922737548672932719152106443920061291755799720371866661675731913255469257125424287184505694891152405574139340841454739472853293973706390630637496605480978774509790631991009834172740796006566593246195359473453273928106881609424766258644698396670056407157306479428120709099340936631042144966901737531061621879188420460326201537441250

---
## Håstad broadcast
_Small $e$ attack._




In [6]:
# set-up: operations performed by the sender

flag_bytes = 32
flag = new_flag(flag_bytes)
f = int.from_bytes(bytes(flag, 'ascii'), byteorder='big')

e=3
recipients = 3
key_bits = 1024

n = []
c = []

for i in range(recipients):
    (_,modulus,_) = new_key(key_bits, e=e)
    n.append(modulus)
    ciphertext = encrypt(f, e, modulus)
    c.append(ciphertext)

In [8]:
# attack: operations performed after intercepting ciphertexts c_i, knowing public keys n_i
import gmpy2
N = math.prod(n)
x = 0

for i in range(recipients):
    Ni = N//n[i]
    x += Ni*inverse(Ni, n[i])
r = gmpy2.iroot_rem(x,e)

In [11]:
# check:
print(f'computed plaintext: {r}')
print(f'original plaintext: {f}')

computed plaintext: (mpz(97289280563237334289352317071707027958633200003166873586132662157624342167928411257598642021172735851973310964074689373699828603426835547098393384939628228133739285214826280326647005646196319646811042020721733065812942158777817644635445272664524364013366717043078995935924995372340326812614025746596813997808), mpz(7127904476302586828237624852375189579983564602931485094518096218656213147499069508350872180839747835119086867981668557156357887557433589249723002448154905357318865620707801991752092370621550464842762717708176425914790515665651504642141917990181750783660427283551894445573158578789850241290726100330254603467617367117743771502694958423171030305334413002392225194857334334088336482316642227315084732067863839235298425622779475492265316626188483635361858577701192661132325609969846612583618820703705626217877059020367605130064348932559061800951696403100238539046685498020928157067335445951347052312826762444266211977808))
original plaintext: 445129472273227971944

---
## malleability
_Oracle attack_

---
## LSB oracle
_Oracle attack_

In [16]:
def oracle_lsb(c):
    m = decrypt(c, d, n)
    lsb = m%2
    return lsb

key_bits = 16
(e,n,d) = new_key(key_bits, e=17)
flag_bytes = 2
flag = new_flag(flag_bytes)
f = int.from_bytes(bytes(flag, 'ascii'), byteorder='big')
c = encrypt(f, e, n)

U = n
L = 0
prophecy=[]
for i in range(key_bits):
    g = encrypt(2**i, e, n)
    o = oracle_lsb(g*c)
    prophecy.append(o)
    if o == 0:
        U = (U+L) // 2
    else:
        L = (U+L) // 2
print(f'bit sequence: {prophecy}')
print(f'Final upper bound: {U}')
print(f'original flag: {flag}, int: {f}')

bit sequence: [1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0]
Final upper bound: 11120
original flag: in, int: 26990
